In [1]:
from pathlib import Path
import os

ROOT = Path("/Users/Admin/Desktop/PARA-OS/01_Projects/Learning-Design-ML-Systems/Data-augmentation-DMLS")

if ROOT.is_dir():
    print(f"ROOT directory exists hence changing directory to '{ROOT.name}'")
    os.chdir(ROOT)
else:
    raise NotADirectoryError

ROOT directory exists hence changing directory to 'Data-augmentation-DMLS'


References - 
- [1] NLPAUG - https://pypi.org/project/nlpaug/
- [2] NLPAUG Cookbook - https://github.com/makcedward/nlpaug/blob/master/example/textual_augmenter.ipynb

In [2]:
import nltk # noqa
from nltk.corpus import wordnet # noqa

def replace_synonyms(sentence):
    words = nltk.word_tokenize(sentence)
    new_words = []
    for word in words:
        synonyms = wordnet.synsets(word)
        if synonyms and synonyms[0].lemmas():
            new_words.append(synonyms[0].lemmas()[0].name())
        else:
            new_words.append(word)
    return " ".join(new_words)

sentence = "The quick brown fox jumps over the lazy dog"
new_sentence = replace_synonyms(sentence)
print(new_sentence)

The quick brown fox jump over the lazy dog


In [3]:
import nlpaug.augmenter.word as naw # noqa

text = 'The quick brown fox jumps over the lazy dog .'
print(text)

The quick brown fox jumps over the lazy dog .


In [4]:
aug = naw.SynonymAug(aug_src='wordnet')
augmented_text = aug.augment(text)
print("Original:")
print(text)
print("Augmented Text:")
print(augmented_text)

Original:
The quick brown fox jumps over the lazy dog .
Augmented Text:
['The quick brown fox jumps ended the indolent heel.']


In [6]:
aug = naw.SynonymAug(aug_src='wordnet')
print("Original:")
print(text)
for idx in range(1,6):
    print()
    augmented_text = aug.augment(text)
    print(f'Augmented Text Sample {idx:02d}:-')
    print(augmented_text[0])
    print("----")

Original:
The quick brown fox jumps over the lazy dog .

Augmented Text Sample 01:-
The ready robert brown slyboots jumps over the lazy dog.
----

Augmented Text Sample 02:-
The quick brown charles james fox leap over the otiose dog.
----

Augmented Text Sample 03:-
The agile brown slyboots jumps over the lazy heel.
----

Augmented Text Sample 04:-
The quick john brown fox jumps concluded the lazy detent.
----

Augmented Text Sample 05:-
The quick brownness george fox jumps concluded the lazy dog.
----
